In [1]:
from lib.File import File
from matplotlib import pyplot as plt
from data.loader import get_kasios_obs, get_obs, map_path
import numpy as np

from math import sqrt
import pandas as pd

In [219]:
# Loads files and makes conversion if necessary
df_all = get_obs(songs = True)
df_kasios = get_kasios_obs(songs = True)

Loading wav files...
Done.
Loading wav files...
Done.


In [220]:
#X = get_spectres(df_all) # Can be very long
#X.to_pickle("data/all_birds_freq.pickle") # Save to cache
X = pd.read_pickle("data/all_birds_freq.pickle") # Quicker to use cache ;

X.head()

1900.0    1910.0    1920.0    1930.0    1940.0    1950.0    1960.0  \
File ID                                                                         
402254  -0.777891 -0.398350 -0.014216 -0.855008  2.075899 -0.221931 -0.655440   
406171  -4.162661 -4.180349 -4.229848 -4.230835 -4.269890 -4.245544 -4.282711   
405901  -0.580728 -0.633019 -0.707576 -0.645468 -0.603866 -0.725716 -0.672938   
405548  -3.228091 -3.251748 -3.226915 -3.256686 -3.309930 -3.233574 -3.255813   
401782  -3.740562 -3.741388 -3.765818 -3.748055 -3.741544 -3.737901 -3.745398   

           1970.0    1980.0    1990.0    ...       5900.0    5910.0    5920.0  \
File ID                                  ...                                    
402254  -1.191778  0.585549  0.236256    ...     0.433391  0.306798 -1.075585   
406171  -4.257561 -4.243178 -4.282526    ...    -1.510778 -1.968467 -1.824303   
405901  -0.657233 -0.662175 -0.732691    ...    -0.300444 -0.458147 -0.586822   
405548  -3.283560 -3.262860 -3.241711    ...     7.121533  6.845443  6.915710   
401782  -3.741317 -3.749377 -3.750319    ...    -2.814041 -2.563670 -2.758136   

           5930.0    5940.0    5950.0    5960.0    5970.0    5980.0    5990.0  
File ID                                                                        
402254  -0.452546 -0.197668 -0.996424 -0.379281 -0.244899 -0.400091 -0.611824  
406171  -1.519034 -1.865900 -1.872201 -1.824038 -1.760304 -1.728515 -1.927553  
405901  -0.149122 -0.337683 -0.258401 -0.292191 -0.543218 -0.267194 -0.134089  
405548   6.386549  5.541339  4.538464  2.270188  0.207980  1.365019  3.283181  
401782  -2.482101 -2.802750 -2.417855 -2.802082 -2.927259 -2.785790 -2.790771  

[5 rows x 410 columns]

In [221]:
#X_kasios = get_spectres(df_kasios) # Can be very long
#X_kasios.to_pickle("data/kasios_birds_freq.pickle")
X_kasios = pd.read_pickle("data/kasios_birds_freq.pickle") # Quicker to use cache ;)

X_kasios.head()

1900.0     1910.0     1920.0     1930.0     1940.0     1950.0  \
ID                                                                     
1   -3.529268  -3.329053  -3.513426  -3.523270  -3.362683  -3.481535   
2   -7.770975  -7.789180  -7.807659  -7.812153  -7.826096  -7.787444   
3  -10.823887 -10.997442 -10.907166 -10.946253 -10.891168 -10.957196   
4   -7.564831  -7.604928  -7.853263  -7.667681  -7.446895  -7.664611   
5   13.974970  13.250154  12.210977  12.134556  11.516458  11.235900   

       1960.0     1970.0     1980.0     1990.0    ...       5900.0    5910.0  \
ID                                                ...                          
1   -3.410325  -3.471149  -3.358526  -3.415705    ...    -3.231955 -3.377782   
2   -7.798715  -7.757632  -7.766777  -7.717729    ...     2.070107  1.240989   
3  -11.014657 -10.992509 -10.977279 -11.082591    ...    -7.309365 -7.188007   
4   -7.868092  -7.727359  -7.579639  -7.770928    ...    -6.860031 -6.743433   
5   10.998921  10.493838  10.029321   9.747640    ...    -3.994871 -3.862566   

      5920.0    5930.0    5940.0    5950.0    5960.0    5970.0    5980.0  \
ID                                                                         
1  -3.150853 -3.322742 -3.141581 -3.317984 -3.318084 -3.201280 -3.373770   
2   0.342733  1.253810 -0.095410  0.345641 -0.313179 -0.376685 -0.871546   
3  -8.339784 -6.470934 -6.471953 -7.001865 -8.281127 -7.377273 -7.894899   
4  -6.865163 -6.472720 -6.746985 -6.491604 -6.479721 -6.478345 -6.351524   
5  -3.970654 -4.009346 -3.824077 -4.031627 -3.842823 -3.883048 -4.091320   

      5990.0  
ID            
1  -3.249895  
2  -0.830625  
3  -7.307045  
4  -6.770690  
5  -3.786353  

[5 rows x 410 columns]

## Learning

In [222]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix

In [223]:
df_all.English_name.unique()

array(['Rose-crested Blue Pipit', 'Blue-collared Zipper', 'Bombadil',
       'Broad-winged Jojo', 'Canadian Cootamum',
       'Carries Champagne Pipit', 'Darkwing Sparrow',
       'Eastern Corn Skeet', 'Green-tipped Scarlet Pipit',
       'Lesser Birchbeere', 'Orange Pine Plover', 'Ordinary Snape',
       'Pinkfinch', 'Purple Tooting Tout', 'Qax', 'Queenscoat',
       'Bent-beak Riffraff', 'Scrawny Jay', 'Vermillion Trillian'],
      dtype=object)

In [224]:
Y = df_all.English_name == "Rose-crested Blue Pipit"

## Fitting

In [225]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33)

In [226]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7)

In [227]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Résultats

In [228]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

In [229]:
print("Confusion sur le train : (Overfit)")
print(confusion_matrix(y_train, y_pred_train))

print("\n")

print("Confusion sur le test :")
print(confusion_matrix(y_test, y_pred_test))

print()

Confusion sur le train : (Overfit)
[[1244    1]
 [  14  125]]


Confusion sur le test :
[[628   8]
 [ 40   7]]



In [230]:
print(accuracy_score(y_train, y_pred_train))
print("Accuracy: %s" % accuracy_score(y_test,y_pred_test))

0.9891618497109826
Accuracy: 0.9297218155197657


In [231]:
y_kasios = clf.predict(X_kasios)

In [232]:
print("Classification with Random Forest \n")
for i in range(len(y_kasios)):
    print("Kasios song n°%s : %s"% (i+1, y_kasios[i]))

Classification with Random Forest 

Kasios song n°1 : False
Kasios song n°2 : True
Kasios song n°3 : False
Kasios song n°4 : False
Kasios song n°5 : False
Kasios song n°6 : False
Kasios song n°7 : False
Kasios song n°8 : False
Kasios song n°9 : False
Kasios song n°10 : False
Kasios song n°11 : False
Kasios song n°12 : False
Kasios song n°13 : False
Kasios song n°14 : False
Kasios song n°15 : False


In [233]:
n_iters = 15
for i in range(n_iters):
    clf.fit(X_train, y_train)
    y_sample = clf.predict(X_kasios)
    y_kasios = [a or b for a, b in zip(y_sample, y_kasios)]

print("Classification with Random Forest after %s iterations\n" % (n_iters))
for i in range(len(y_kasios)):
    print("Kasios song n°%s : %s"% (i+1, y_kasios[i]))

Classification with Random Forest after 15 iterations

Kasios song n°1 : False
Kasios song n°2 : True
Kasios song n°3 : False
Kasios song n°4 : False
Kasios song n°5 : False
Kasios song n°6 : False
Kasios song n°7 : False
Kasios song n°8 : False
Kasios song n°9 : False
Kasios song n°10 : False
Kasios song n°11 : False
Kasios song n°12 : False
Kasios song n°13 : True
Kasios song n°14 : False
Kasios song n°15 : False
